In [1]:
print("Ok")

Ok


In [2]:
%pwd

'f:\\2024\\python\\deeplearning\\chatbot\\End-to-end-Medical-Chatbot-Generative-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'f:\\2024\\python\\deeplearning\\chatbot\\End-to-end-Medical-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [11]:
#text_chunks

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
import os
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "120"  # Increase timeout to 120 seconds

#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\mahab\AppData\Local\Temp\ipykernel_19048\1701744049.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\mahab\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
# query_result

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_5D4Hk4_7t8cxb6JZ2GdwsW3GDRUYT5rrcEirVABA37R8yMBKE27Tf5o9Ctqvbbx8JU1KxP'

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [21]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [22]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [23]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("What is Acne?")

In [27]:
retrieved_docs

[Document(id='39487f78-c1a4-461a-8908-4f46d053eaf7', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='54fb25b8-a705-4f5c-ae0c-6979757644f5', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [28]:
# Assuming `retrieved_docs` is the list of documents returned by the retriever
answer = retrieved_docs[0].page_content.split("\n")[-1].strip()
print(answer)

GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26


In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

In [56]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama  # Import Ollama from langchain_community

# Define the system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved information to answer the questions given "
    "If there is no the answer or any hint from the retrieved information, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [57]:
# Initialize the Ollama model
MODEL = "deepseek-r1:1.5b"
llm = Ollama(model=MODEL)

# Create the document chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)


In [50]:
# Create the retrieval chain
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [70]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama

# Updated system prompt to enforce strict context usage
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use **only** the following pieces of retrieved context to answer "
    "the question. If the context does not provide enough information, "
    "or if the question is unrelated to the context, say 'I don't know.' "
    "Do not use any external knowledge or make up answers."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Initialize the Ollama model
MODEL = "deepseek-r1:1.5b"
llm = Ollama(model=MODEL)

# Create the document chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Function to validate context and invoke the chain
def ask_question(question):
    # Retrieve context
    retrieved_context = retriever.invoke(question)
    
    # Check if the retrieved context is empty or irrelevant
    if not retrieved_context or all(doc.page_content.strip() == "" for doc in retrieved_context):
        return "I don't know."
    
    # Invoke the RAG chain with the question
    response = rag_chain.invoke({"input": question})
    return response["answer"]

# Example usage
response = ask_question("pneumonia?")
print(response)

KeyboardInterrupt: 

In [69]:
import re


def remove_think_tags(text):
    cleaned_text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

cleaned_response = remove_think_tags(response)
print(cleaned_response)

The symptoms of bronchiectasis include:

1. **Cough**: The primary manifestation of chronic lung disease, often appearing sudden and severe.

2. **Sputum production**: The body's response to infection, which may be sputful (mixing mucus and pus), sometimes bloody in some cases.

3. **Wheezing and shortness of breath**: Symptoms of infection that worsens the cough and can lead to increased fever.

4. **Infection level**: A constant, low-level of infection that can flare, causing worsening symptoms like cough and fever.

5. **Infected lung area**: The affected bronchial tube may become severely infected, leading to pneumonia-like severity.

6. **Lung scarring after prolonged use of a ventilator**: This can improve in the months following, though it may not fully cure the disease.

7. **Pneumonia**: A severe form of lung infection that is often life-threatening and can kill 90% of ARDS patients with prompt treatment, while modern treatments increase survival rates to less than half.
